# GA-PLSR for Feature Selection
Hyperspectral images are high-dimensional data sets that contain hundreds or thousands of spectral bands. However, not all of these bands are useful for analysis or modeling. In fact, including too many bands in a model can lead to overfitting and poor performance. Therefore, it is essential to perform feature selection to identify the most informative spectral bands for a given application.

One approach to feature selection is **partial least squares regression (PLSR)**, which is a multivariate regression technique that models the relationship between a set of predictor variables (spectral bands) and a response variable (e.g., vegetation index, mineral abundance, etc.). PLSR can also be used for dimensionality reduction, as it projects the high-dimensional data onto a lower-dimensional space that captures the most relevant information.

However, **selecting the optimal subset of spectral bands for PLSR can be a challenging task**, especially when dealing with a large number of variables. One solution is to use a **genetic algorithm (GA)** to search for the best subset of variables that maximizes the predictive performance of the PLSR model. This approach is known as GA-PLSR, and it has been shown to be effective for feature selection in hyperspectral images.

The basic idea behind GA-PLSR is to create a population of candidate subsets of spectral bands (i.e., chromosomes), evaluate their performance using a fitness function (e.g., cross-validation error), and then use genetic operators (e.g., mutation, crossover) to generate new candidate solutions. The process is repeated for several generations until a stopping criterion is met (e.g., maximum number of iterations).

## **Using the Notebook**

1. Enter the name of the target column in your DataFrame. This is the column that you want to predict using your PLSR model. Specify the required parameters for the GA-PLSR algorithm. These include the number of variables to select (n_components), the number of chromosomes in the population (pop_size), and the number of generations (n_gen) to run the algorithm.

2. Load your data file (CSV) using the provided function *load_data()*. This function takes the path to your CSV file as input and returns a pandas DataFrame containing your data.

2. Split to Train/ Test using *split_data()* function.

4. Run the GA-PLSR algorithm, which takes your data DataFrame, target column name, and algorithm parameters as inputs. This function returns a pandas DataFrame containing the relevant features after dimensionality reduction.
 
 🚩🚩 **This part can take time** (depents on your data size).


5. Use the selected features to train your PLSR model with *Fit_PLSR* function

⚡⚡ **Important**: Typically, GA-PLSR is applied to the training set rather than the entire dataset to prevent overfitting. In this notebook, the data is also split during the process, and GA-PLSR is applied only to the training set.

**Credit**:  [Github](https://github.com/hkaneko1985/gapls_gasvr/blob/master/demo_gapls.py)

## **Functions**

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
!pip install deap
from deap import base
from deap import creator
from deap import tools
from sklearn import datasets
from sklearn import model_selection
from sklearn.cross_decomposition import PLSRegression
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
def load_data(csv_path, feature_col_start, feature_col_end, target_col):
    """
    Load a CSV file into a Pandas DataFrame,drop Nan, and separate the feature and target columns.

    Parameters:
        csv_path (str): Path to the CSV file to load.
        feature_col_start, feature_col_end, (ints): Range of column indices to use as features.
        target_col (str or int): Name or index of the column to use as target.

    Returns:
        new_df: A df containing the features + labels DataFrame.
    """
    # Load CSV into a Pandas DataFrame
    df = pd.read_csv(csv_path)

    # drop nan
    df = df.dropna()

    # Extract the feature and target columns
    new_df = df[df.columns[feature_col_start: feature_col_end]]
    new_df[target_col] = df[target_col]

    return new_df

In [31]:
def split_data(df, target_col, test_size=0.3, random_state=42):
    """
    Splits the input DataFrame into training and testing sets.
    
    Parameters:
    -----------
    df (pandas DataFrame): The input DataFrame containing the features and target variable.
    target_col (str): The name of the target column in the DataFrame.
    test_size (float, optional): The proportion of the data to use for testing (default=0.3).
    random_state (int, optional): The random seed to use for the train-test split (default=42).
        
    Returns:
    --------
    X_train (pandas DataFrame): The training set features.     
    X_test (pandas DataFrame): The testing set features.        
    y_train (pandas Series): The training set target variable.
    y_test (pandas Series): The testing set target variable.
    """
    # Extract the features and target variable from the DataFrame
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Return the training and testing sets
    return X_train, X_test, y_train, y_test

In [32]:
def Fit_PLSR(X_train_sort,X_test_sort,y_train):
  """
    Fit PLSR on the new data after feature selection using GA-PLSR.
    
    Parameters:
    -----------
    X_train_sort (pandas DataFrame): contain the features that GA-PLSR keep
    X_test_sort (pandas DataFrame): contain the features that GA-PLSR keep

    Returns:
    --------
    X_train_plsr (pandas DataFrame):Data after appling PLSR.     
    X_test_plsr (pandas DataFrame):Data after appling PLSR.        
    
    You can use this datasets as train/test set for ML models.
    """

  bands_list = selected_X_variable_numbers.tolist()

  # Fit PLSR model
  n_bands = len(X_train_sort.columns)
  plsr = PLSRegression(n_components=n_bands)
  plsr.fit(X_train_sort, y_train)

  # Transform the training and testing data using the PLSR model
  X_train_plsr = plsr.transform(X_train_sort)
  X_test_plsr = plsr.transform(X_test_sort)

  return X_train_plsr, X_test_plsr


In [55]:
def merge_dataframes(X_train, X_test, y_train, y_test):
    """
    Merge X_train, X_test, y_train, and y_test into a single dataframe.
    """
    # Convert the numpy arrays to pandas dataframes
    X_train_df = pd.DataFrame(X_train)
    X_test_df = pd.DataFrame(X_test)
    y_train_df = pd.DataFrame(y_train)
    y_test_df = pd.DataFrame(y_test)

    # Merge X_train and X_test into a single DataFrame
    X = pd.concat([X_train_df, X_test_df], ignore_index=True)

    # Merge y_train and y_test into a single DataFrame
    y = pd.concat([y_train_df, y_test_df], ignore_index=True)

    # Concatenate X and y into a single DataFrame
    df = pd.concat([X, y], axis=1)

    return df

## Example

In [33]:
# Define input parameters
csv_path = '/content/data.csv'
feature_idx_i,feature_idx_f = 16,-2
target_col = 'A'

# GA-PLSR parameters
number_of_population = 100
number_of_generation = 10
max_number_of_components = 10
fold_number = 5
probability_of_crossover = 0.5
probability_of_mutation = 0.2
threshold_of_variable_selection = 0.5

In [34]:
# Load data
data = load_data(csv_path, feature_idx_i,feature_idx_f, target_col)
data.head()

<ipython-input-30-629dbe82679f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[target_col] = df[target_col]


,397.32,400.2,403.09,405.97,408.85,411.74,414.63,417.52,420.4,423.29,...,978.88,981.96,985.05,988.13,991.22,994.31,997.4,1000.49,1003.58,A
0,0.179808,0.152106,0.129191,0.115715,0.107613,0.102074,0.101501,0.099727,0.096248,0.096929,...,0.458213,0.464172,0.458520,0.462214,0.467727,0.467549,0.466043,0.471523,0.447471,2.017270
1,0.221156,0.186298,0.160032,0.146194,0.136323,0.128331,0.124891,0.121850,0.116359,0.114495,...,0.717970,0.717748,0.722268,0.726763,0.738159,0.741649,0.739217,0.762054,0.622104,1.872474
2,0.221893,0.185626,0.164002,0.154074,0.146511,0.137888,0.133002,0.130920,0.128935,0.126446,...,0.670528,0.675308,0.669332,0.689363,0.685825,0.698885,0.689815,0.705207,0.580815,2.043818
3,0.162126,0.129779,0.104428,0.089685,0.080833,0.075142,0.068085,0.063978,0.058188,0.054447,...,0.570670,0.574177,0.580435,0.579218,0.582644,0.592902,0.597743,0.609343,0.480618,2.123489
4,0.206857,0.164631,0.137415,0.118823,0.102912,0.097850,0.090029,0.084146,0.077650,0.072445,...,0.602451,0.609186,0.624415,0.622750,0.633371,0.640970,0.649146,0.659158,0.536100,2.122085


In [35]:
X_train, X_test, y_train, y_test = split_data(data, target_col, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((429, 204), (184, 204), (429,), (184,))

In [43]:
# autoscaling
autoscaled_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std(ddof=1)

# convert to numpy
autoscaled_X_train  = autoscaled_X_train.values
autoscaled_y_train = autoscaled_y_train.values

    # GAPLS
creator.create('FitnessMax', base.Fitness, weights=(1.0,))  # for minimization, set weights as (-1.0,)
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
min_boundary = np.zeros(X_train.shape[1])
max_boundary = np.ones(X_train.shape[1]) * 1.0


def create_ind_uniform(min_boundary, max_boundary):
    index = []
    for min, max in zip(min_boundary, max_boundary):
        index.append(random.uniform(min, max))
    return index


toolbox.register('create_ind', create_ind_uniform, min_boundary, max_boundary)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.create_ind)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)


def evalOneMax(individual):
    individual_array = np.array(individual)
    selected_X_variable_numbers = np.where(individual_array > threshold_of_variable_selection)[0]
    selected_autoscaled_X_train = autoscaled_X_train[:, selected_X_variable_numbers]
    if len(selected_X_variable_numbers):
        # cross-validation
        pls_components = np.arange(1, min(np.linalg.matrix_rank(selected_autoscaled_X_train) + 1,
                                          max_number_of_components + 1), 1)
        r2_cv_all = []
        for pls_component in pls_components:
            model_in_cv = PLSRegression(n_components=pls_component)
            estimated_y_train_in_cv = np.ndarray.flatten(
                model_selection.cross_val_predict(model_in_cv, selected_autoscaled_X_train, autoscaled_y_train,
                                                  cv=fold_number))
            estimated_y_train_in_cv = estimated_y_train_in_cv * y_train.std(ddof=1) + y_train.mean()
            r2_cv_all.append(1 - sum((y_train - estimated_y_train_in_cv) ** 2) / sum((y_train - y_train.mean()) ** 2))
        value = np.max(r2_cv_all)
    else:
        value = -999

    return value,


toolbox.register('evaluate', evalOneMax)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

# random.seed(100)
random.seed()
pop = toolbox.population(n=number_of_population)

print('Start of evolution')

fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

print('Start of evolution')

for generation in tqdm(range(number_of_generation)):
    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < probability_of_crossover:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < probability_of_mutation:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop[:] = offspring
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean ** 2) ** 0.5

    tqdm.write('-- Generation {0} --'.format(generation + 1))
    tqdm.write('  Evaluated %i individuals' % len(invalid_ind))
    tqdm.write('  Min %s' % min(fits))
    tqdm.write('  Max %s' % max(fits))
    tqdm.write('  Avg %s' % mean)
    tqdm.write('  Std %s' % std)

tqdm.write('-- End of (successful) evolution --')

best_individual = tools.selBest(pop, 1)[0]
best_individual_array = np.array(best_individual)
selected_X_variable_numbers = np.where(best_individual_array > threshold_of_variable_selection)[0]
# print('Selected variables : %s, %s' % (selected_X_variable_numbers, best_individual.fitness.values))


# Collect all importance features and update the datasets:
bands_list = selected_X_variable_numbers.tolist()
X_train_sort = X_train[X_train.columns[bands_list]]
X_test_sort = X_test[X_test.columns[bands_list]]

print(X_train_sort.shape,X_test_sort.shape)

/usr/local/lib/python3.8/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.8/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Start of evolution
Start of evolution


  0%|          | 0/10 [00:00<?, ?it/s]

-- Generation 1 --
  Evaluated 58 individuals
  Min 0.23575674507751143
  Max 0.2576324566481273
  Avg 0.2461146859067201
  Std 0.0039295782067678975
-- Generation 2 --
  Evaluated 52 individuals
  Min 0.24005234809565024
  Max 0.2846703777525629
  Avg 0.2494962626454223
  Std 0.005564975628055298
-- Generation 3 --
  Evaluated 53 individuals
  Min 0.2423248625951514
  Max 0.2846703777525629
  Avg 0.253747199700636
  Std 0.007542943310906579
-- Generation 4 --
  Evaluated 63 individuals
  Min 0.2405732818361862
  Max 0.2874884411688723
  Avg 0.2580630714479994
  Std 0.01031650689752442
-- Generation 5 --
  Evaluated 53 individuals
  Min 0.2431845862896712
  Max 0.2874884411688723
  Avg 0.2667766528916663
  Std 0.01088105773231832
-- Generation 6 --
  Evaluated 66 individuals
  Min 0.2515239745461382
  Max 0.2878776173584695
  Avg 0.2737610781580834
  Std 0.009025876639099505
-- Generation 7 --
  Evaluated 50 individuals
  Min 0.26186102962750113
  Max 0.2878776173584695
  Avg 0.2812673

In [45]:
X_train_plsr, X_test_plsr = Fit_PLSR(X_train_sort,X_test_sort,y_train)
X_train_plsr.shape,X_test_plsr.shape

((429, 73), (184, 73))

In [58]:
# Merged all the procceced data again to new df - Optional
merged_df = merge_dataframes(X_train_plsr, X_test_plsr, y_train, y_test)

# Save
# merged_df.to_csv("name_to_save.csv")